<div class='container2'>
    <hr style="height:10px"> 
    <h1 ALIGN='center'>Laboratório de Inovação e Automação 1 (LIA 1)</h1>
    <h2 ALIGN='center'>Tema: Visao computacional</h2>
    <h3 ALIGN='center'>Autores: Gustavo Mota Barros e Pedro Ferreira Galvao Neto</h3>
    <hr style="height:10px">
    <img src='https://sp-ao.shortpixel.ai/client/to_auto,q_lossy,ret_img/https://blog.grvppe.com/wp-content/uploads/2019/12/Computer-Vision-pixforce-drone-1280x640.jpg' ALIGN='center' width="800">
    <hr style="height:10px">
</div>
<div style="padding-left:20px; padding-right:20px;">
    <p style="font-size:16px; white">
        O projeto desenvolvido consiste na utilizaçao de modelos de inteligencia artificial voltados a area de visao computacional, para o reconhecimento de objetos. Este projeto demonstrará como o modelo se comporta em relacao a imagens e videos, alem de aplicaçoes que demonstram a grande utilidade destes modelos.
    </p>
</div>
        

# Aplicaçoes com o modelo de visao computacional YOLO V4

## Passos envolvidos:

### Baixando os arquivos necessarios :

   Primeiro, como utilizaremos o modelo yolo v4 para o reconhecimento de imagens e videos com a biblioteca cv2 do openCV, devemos baixar os arquivos yolov4.weights e yolov4.cfg que sao disponibilizados no seguinte repositorio: https://github.com/AlexeyAB/darknet
   Também, devemos baixar, neste repositorio, o arquivo coco.names que se encontra na pasta cfg

### Carregando o modelo :

In [1]:
#Primeiro, deve-se importar as seguintes bibliotecas
import cv2
import time
import numpy as np
import random as rd
import pafy
import os
import csv

#### Carregando o modelo completo para a detecçao em imagens:

In [2]:
# carregando a rede neural
net = cv2.dnn.readNet('yolov4.weights', 'yolov4.cfg')

# criando o modelo a partir da rede neural
model = cv2.dnn.DetectionModel(net)

# setando os parametros de entrada para o modelo utilizado
model.setInputParams(size=(608,608), scale=(1/255))

#### Carregando o modelo tiny para a detecçao em videos :

In [2]:
# carregando a rede neural
net = cv2.dnn.readNet('yolov4-tiny.weights', 'yolov4-tiny.cfg')

# criando o modelo a partir da rede neural
model = cv2.dnn.DetectionModel(net)

# setando os parametros de entrada para o modelo utilizado
model.setInputParams(size=(416,416), scale=(1/255))

### Carregando o arquivo de nomes das imagens reconhecidas pelo modelo :

In [3]:
# carregando os nomes para uma lista
class_names = []
with open('coco_names.txt','r') as f:
    class_names = [cname.strip() for cname in f.readlines()]

### Configurando as cores para cada classe :

In [4]:
# definindo as cores de cada nome
name_color = []
for i in class_names:
    color =(((sum(map(ord,i))*2)%255),((sum(map(ord,i))*3)%255),((sum(map(ord,i))*5)%255))
    name_color.append(color)

### Detectando objetos em Imagens :

In [17]:
# Primeiro devemos coletar a imagem a ser detectada
#iremos usar o duckduckgo para isso

from duckduckgo_search import ddg_images
import requests

def search_img(img_name, num_img=15):
    ddg_img = ddg_images(img_name, max_results=num_img)
    img_list = [dicio['image'] for dicio in ddg_img]
    return img_list

img_name = input("Digite em ingles o nome do objeto que deseja testar:")

if img_name not in class_names:
    print("O modelo nao é capaz de reconhecer este objeto. Tente novamente!")
else:
    images = search_img(img_name)
    img_list = []
    for i in images:
        print("Carregando links validos...")
        try:
            if requests.get(i).status_code in range(400,500):
                images.remove(i)
            else:
                response = requests.get(i)
                img_array = np.array(bytearray(response.content), dtype=np.uint8)
                img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
                res_img = cv2.resize(img,(416,416))
                img_list.append(res_img)
        except:
            images.remove(i)

Digite em ingles o nome do objeto que deseja testar:person
Carregando links validos...
Carregando links validos...
Carregando links validos...
Carregando links validos...
Carregando links validos...
Carregando links validos...
Carregando links validos...
Carregando links validos...
Carregando links validos...
Carregando links validos...
Carregando links validos...
Carregando links validos...


In [19]:
# agora iremos realizar a detecçao na imagem

for i in img_list:
    model_image = i.copy()

    classes, scores, boxes = model.detect(model_image , 0.1, 0.2)

    for (classid, score, box) in zip(classes, scores, boxes):
        color = name_color[classid]
        label = f"{class_names[classid]}: {score}"

        cv2.rectangle(model_image, box, color, 2)
        cv2.putText(model_image, label, (box[0], box[1] + 15), cv2.FONT_HERSHEY_COMPLEX, 0.5, color, 2)
        print(f"class: {class_names[classid]} ---> score: {score}")

    cv2.imshow("detections", model_image)
    cv2.waitKey(0)

class: person ---> score: 0.4117244482040405
class: tie ---> score: 0.22676917910575867
class: person ---> score: 0.9023652672767639
class: tie ---> score: 0.6395965814590454
class: person ---> score: 0.9572144746780396
class: tie ---> score: 0.7818458080291748
class: person ---> score: 0.861284077167511
class: person ---> score: 0.9307529330253601
class: person ---> score: 0.8180352449417114
class: person ---> score: 0.951819121837616
class: tie ---> score: 0.9398318529129028


### Detectando objetos em videos :

In [6]:
''' Para que o recurso de videos do youtube funcione, é necessario instalar a biblioteca youtube-dl e modificar o arquivo 
youtube.py que se encontra na pasta extractor, indo na linha 1794 e comentando-a. Apos isso, acesse a biblioteca pafy e comente
as linhas 53, 54 e 55 do arquivo backend_youtube_dl.py '''

from tqdm.notebook import tqdm

url = "https://www.youtube.com/watch?v=iBlRm6BzgqY&t=904s&ab_channel=LucasSousa" #basta colocar o url do seu video aqui
videoPafy = pafy.new(url)

def getNormalQuality(videoPafy):
    for stream in videoPafy.videostreams:
        if stream.resolution == '1280x720' and stream.extension =='mp4':
            return stream
    return videoPafy.getbest(preftype='mp4')

cap = cv2.VideoCapture(getNormalQuality(videoPafy).url)
filename = 'test_video.mp4'
codec = cv2.VideoWriter_fourcc(*'XVID')
test_out = cv2.VideoWriter(filename, codec, 30, (1280, 720))

duration = 180*1000 # duraçao do video em milissegundos
pbar = tqdm(total=duration , desc="Carregando Video: ")
n = 0
k = 0
while True:

    _, frame = cap.read()
    test_out.write(frame)
    
    vid_time = cap.get(cv2.CAP_PROP_POS_MSEC)
    n = vid_time - k
    k = vid_time
    pbar.update(n)
    
    if vid_time >= duration or not _:
        break

test_out.release()
cv2.destroyAllWindows()
cap.release()

Carregando Video:   0%|          | 0/180000 [00:00<?, ?it/s]

In [7]:
cap = cv2.VideoCapture('test_video.mp4')

In [8]:
# Para que a fluidez do video seja mantida iremos escrever um arquivo mp4 com as detecçoes ao inves de faze-la
# simultaneamente

codec = cv2.VideoWriter_fourcc(*'XVID')
vid_out = cv2.VideoWriter('result.mp4', codec, 30, (1280, 720))

file = open("detec.csv", 'w', newline='', encoding= 'utf-8')
writer = csv.writer(file)
writer.writerow(['Frame', 'Objeto', 'Score', 'x_pos', 'y_pos'])

while cap.isOpened():

    _, f = cap.read()
    start = time.time()
    classes, scores, boxes = model.detect(f,0.1,0.2)
    end = time.time()

    for (classid, score, box) in zip(classes, scores, boxes):
        color = name_color[classid]
        label = f"{class_names[classid]}: {score}"
        writer.writerow([cap.get(cv2.CAP_PROP_POS_MSEC), class_names[classid], score, (box[0] + int(box[2]/2)), 
                        (box[1] + int(box[3]/2))])
        
        cv2.rectangle(f, box, color, 2)
        cv2.putText(f, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, color, 2)
    
    fps_label = f"FPS: {round((1/(end - start)),2)}"

    cv2.putText(f, fps_label, (0,25), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 5)
    cv2.putText(f, fps_label, (0,25), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 3)
    
    vid_out.write(f)
    cv2.imshow("Video Detection",f)
    
    if cv2.waitKey(1) == 27 or not _:
        break

file.close()
vid_out.release()
cap.release()
cv2.destroyAllWindows()

### Detectando objetos na camera :

#### Capturando a webcam do computador:

In [9]:
cap =  cv2.VideoCapture(0) # aqui vai o numero referente ao dispositivo

#### Capturando a camera do celular

In [6]:
cap =  cv2.VideoCapture("http://192.168.1.3:4747/video") # aqui vai o numero referente ao dispositivo

#### Detecçao

In [10]:
#codigo de detecçao
while cap.isOpened():

    _, frame = cap.read()
    start = time.time()
    classes, scores, boxes = model.detect(frame,0.1,0.2)
    end = time.time()

    for (classid, score, box) in zip(classes, scores, boxes):
        color = name_color[classid]
        label = f"{class_names[classid]}: {score}"
        
        cv2.rectangle(frame, box, color, 2)
        cv2.putText(frame, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, color, 2)
    
    fps_label = f"FPS: {round((1/(end - start)),2)}"

    cv2.putText(frame, fps_label, (0,25), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,0), 5)
    cv2.putText(frame, fps_label, (0,25), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 3)

    cv2.imshow("detections", frame)

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()